# Configuration Management

This notebook provides a comprehensive configuration management system for the Criteria Evidence Agent project.
It replaces the scattered Hydra configuration files with an interactive, notebook-based approach.

In [ ]:
import os
import json
import yaml
from pathlib import Path
from typing import Dict, Any, Optional, List
from dataclasses import dataclass, asdict, field
from omegaconf import DictConfig, OmegaConf
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd

## Configuration Classes

Define structured configuration classes for better type safety and validation.

In [ ]:
@dataclass
class DataConfig:
    """Data configuration."""
    root_dir: str = "./Data"
    groundtruth_path: str = "./Data/groundtruth/redsm5_ground_truth.json"
    posts_path: str = "./Data/redsm5/redsm5_posts.csv"
    id_field: str = "post_id"
    text_field: str = "text"
    multi_label_fields: List[str] = field(default_factory=lambda: [
        "ANHEDONIA", "APPETITE_CHANGE", "COGNITIVE_ISSUES", "DEPRESSED_MOOD",
        "FATIGUE", "PSYCHOMOTOR", "SLEEP_ISSUES", "SPECIAL_CASE",
        "SUICIDAL_THOUGHTS", "WORTHLESSNESS"
    ])
    evidence_char_spans_field: Optional[str] = None
    evidence_token_labels_field: Optional[str] = None
    max_length: int = 256
    val_size: float = 0.15
    test_size: float = 0.15
    seed: int = 42

@dataclass
class LoRAConfig:
    """LoRA configuration for parameter-efficient fine-tuning."""
    enabled: bool = False
    r: int = 16
    alpha: int = 32
    dropout: float = 0.05
    target_modules: List[str] = field(default_factory=lambda: ["query", "key", "value"])

@dataclass
class EncoderConfig:
    """Encoder configuration."""
    type: str = "roberta"
    pretrained_model_name_or_path: str = "roberta-base"
    gradient_checkpointing: bool = True
    freeze_encoder: bool = False
    output_dropout: float = 0.2
    pooling: str = "cls"
    layer_norm_eps: float = 1e-12
    lora: LoRAConfig = field(default_factory=LoRAConfig)

@dataclass
class HeadLayersConfig:
    """Classification head layers configuration."""
    hidden_dims: List[int] = field(default_factory=list)
    activation: str = "gelu"
    dropout: float = 0.1

@dataclass
class MultiLabelHeadConfig:
    """Multi-label classification head configuration."""
    type: str = "multi_label"
    labels: List[str] = field(default_factory=list)
    layers: HeadLayersConfig = field(default_factory=HeadLayersConfig)
    classifier_dropout: float = 0.1
    loss: str = "bce"
    label_smoothing: float = 0.0
    pos_weight: List[float] = field(default_factory=list)
    thresholds: Dict[str, float] = field(default_factory=lambda: {
        "ANHEDONIA": 0.5, "APPETITE_CHANGE": 0.5, "COGNITIVE_ISSUES": 0.5,
        "DEPRESSED_MOOD": 0.5, "FATIGUE": 0.5, "PSYCHOMOTOR": 0.5,
        "SLEEP_ISSUES": 0.5, "SPECIAL_CASE": 0.5, "SUICIDAL_THOUGHTS": 0.5,
        "WORTHLESSNESS": 0.5
    })

@dataclass
class TokenHeadConfig:
    """Token classification head configuration."""
    enabled: bool = False
    type: str = "token_classification"
    num_labels: int = 2
    layers: HeadLayersConfig = field(default_factory=HeadLayersConfig)
    classifier_dropout: float = 0.1
    loss: str = "cross_entropy"
    ignore_index: int = -100
    class_weights: List[float] = field(default_factory=list)

@dataclass
class SpanHeadConfig:
    """Span classification head configuration."""
    enabled: bool = False
    type: str = "span_classification"
    layers: HeadLayersConfig = field(default_factory=HeadLayersConfig)
    classifier_dropout: float = 0.1
    loss: str = "cross_entropy"
    ignore_index: int = -100

@dataclass
class ModelHeadsConfig:
    """Model heads configuration."""
    symptom_labels: MultiLabelHeadConfig = field(default_factory=MultiLabelHeadConfig)
    evidence_token: TokenHeadConfig = field(default_factory=TokenHeadConfig)
    evidence_span: SpanHeadConfig = field(default_factory=SpanHeadConfig)

@dataclass
class ModelConfig:
    """Model configuration."""
    encoder: EncoderConfig = field(default_factory=EncoderConfig)
    heads: ModelHeadsConfig = field(default_factory=ModelHeadsConfig)

In [ ]:
@dataclass
class OptimizerConfig:
    """Optimizer configuration."""
    name: str = "adamw"
    learning_rate: float = 2e-5
    weight_decay: float = 0.01
    eps: float = 1e-8
    layerwise_lr_decay: float = 1.0

@dataclass
class SchedulerConfig:
    """Scheduler configuration."""
    name: str = "linear"
    warmup_ratio: float = 0.1
    cosine_cycles: float = 0.5
    onecycle_max_lr: float = 5e-5
    onecycle_pct_start: float = 0.3
    plateau_patience: int = 2
    plateau_factor: float = 0.5
    polynomial_power: float = 1.0

@dataclass
class FocalLossConfig:
    """Focal loss configuration."""
    initial_gamma: float = 2.0
    target_positive_rate: float = 0.25
    alpha: float = 0.25
    min_gamma: float = 1.0
    max_gamma: float = 5.0

@dataclass
class EarlyStoppingConfig:
    """Early stopping configuration."""
    patience: int = 3
    monitor: str = "val_symptom_labels_macro_f1"
    min_delta: float = 0.001

@dataclass
class LossWeightsConfig:
    """Loss weights configuration."""
    symptom_labels: float = 1.0
    evidence_token: float = 1.0
    evidence_span_start: float = 1.0
    evidence_span_end: float = 1.0

@dataclass
class TrainingConfig:
    """Training configuration."""
    batch_size: int = 8
    val_batch_size: int = 16
    test_batch_size: int = 16
    num_workers: int = 8
    gradient_accumulation_steps: int = 1
    max_epochs: int = 100
    optimizer: OptimizerConfig = field(default_factory=OptimizerConfig)
    scheduler: SchedulerConfig = field(default_factory=SchedulerConfig)
    max_grad_norm: float = 1.0
    amp: bool = True
    bf16: bool = True
    logging_interval: int = 50
    ema_decay: float = 0.0
    focal: FocalLossConfig = field(default_factory=FocalLossConfig)
    early_stopping: EarlyStoppingConfig = field(default_factory=EarlyStoppingConfig)
    loss_weights: LossWeightsConfig = field(default_factory=LossWeightsConfig)
    # Checkpoint configuration
    checkpoint_dir: str = "./checkpoints"
    save_every_n_epochs: int = 5
    keep_last_n_checkpoints: int = 3
    auto_resume: bool = True

In [ ]:
@dataclass
class HPOSamplerConfig:
    """HPO sampler configuration."""
    _target_: str = "optuna.samplers.TPESampler"
    seed: int = 42
    multivariate: bool = True

@dataclass
class HPOPrunerConfig:
    """HPO pruner configuration."""
    _target_: str = "optuna.pruners.MedianPruner"
    n_startup_trials: int = 5
    n_warmup_steps: int = 100

@dataclass
class HPOConfig:
    """Hyperparameter optimization configuration."""
    study_name: str = "redsm5_hpo"
    storage: str = "sqlite:///optuna.db"
    direction: str = "maximize"
    n_trials: int = 500
    n_jobs: int = 1
    timeout: Optional[int] = None
    sampler: HPOSamplerConfig = field(default_factory=HPOSamplerConfig)
    pruner: HPOPrunerConfig = field(default_factory=HPOPrunerConfig)
    # Auto-resume configuration
    auto_resume: bool = True
    checkpoint_interval: int = 10  # Save study state every N trials

@dataclass
class MLflowConfig:
    """MLflow configuration."""
    tracking_uri: str = "http://127.0.0.1:5000"
    experiment_name: str = "redsm5_classification"
    nested: bool = False
    autolog: bool = True

@dataclass
class ExperimentConfig:
    """Complete experiment configuration."""
    seed: int = 1337
    data: DataConfig = field(default_factory=DataConfig)
    model: ModelConfig = field(default_factory=ModelConfig)
    training: TrainingConfig = field(default_factory=TrainingConfig)
    hpo: HPOConfig = field(default_factory=HPOConfig)
    mlflow: MLflowConfig = field(default_factory=MLflowConfig)
    
    def __post_init__(self):
        """Post-initialization to set up dependent fields."""
        # Set multi-label fields in model head
        self.model.heads.symptom_labels.labels = self.data.multi_label_fields

## Configuration Manager

A comprehensive configuration manager with save/load capabilities and validation.

In [ ]:
class ConfigurationManager:
    """Manages experiment configurations with save/load and validation capabilities."""
    
    def __init__(self, config_dir: str = "./configs_notebook"):
        self.config_dir = Path(config_dir)
        self.config_dir.mkdir(exist_ok=True)
        self.current_config: Optional[ExperimentConfig] = None
        
    def create_default_config(self) -> ExperimentConfig:
        """Create a default configuration."""
        return ExperimentConfig()
    
    def load_config(self, config_name: str) -> ExperimentConfig:
        """Load configuration from file."""
        config_path = self.config_dir / f"{config_name}.yaml"
        if not config_path.exists():
            raise FileNotFoundError(f"Configuration file not found: {config_path}")
        
        with open(config_path, 'r') as f:
            config_dict = yaml.safe_load(f)
        
        # Convert dict to ExperimentConfig
        self.current_config = self._dict_to_config(config_dict)
        return self.current_config
    
    def save_config(self, config: ExperimentConfig, config_name: str) -> None:
        """Save configuration to file."""
        config_path = self.config_dir / f"{config_name}.yaml"
        config_dict = asdict(config)
        
        with open(config_path, 'w') as f:
            yaml.dump(config_dict, f, default_flow_style=False, indent=2)
        
        print(f"Configuration saved to: {config_path}")
    
    def _dict_to_config(self, config_dict: Dict[str, Any]) -> ExperimentConfig:
        """Convert dictionary to ExperimentConfig."""
        # This is a simplified conversion - in practice, you might want more robust handling
        return ExperimentConfig(**config_dict)
    
    def list_configs(self) -> List[str]:
        """List available configuration files."""
        return [f.stem for f in self.config_dir.glob("*.yaml")]
    
    def validate_config(self, config: ExperimentConfig) -> List[str]:
        """Validate configuration and return list of issues."""
        issues = []
        
        # Validate data paths
        if not Path(config.data.groundtruth_path).exists():
            issues.append(f"Groundtruth file not found: {config.data.groundtruth_path}")
        if not Path(config.data.posts_path).exists():
            issues.append(f"Posts file not found: {config.data.posts_path}")
        
        # Validate model configuration
        if config.model.encoder.type not in ["bert", "roberta", "deberta"]:
            issues.append(f"Unsupported encoder type: {config.model.encoder.type}")
        
        # Validate training parameters
        if config.training.batch_size <= 0:
            issues.append("Batch size must be positive")
        if config.training.optimizer.learning_rate <= 0:
            issues.append("Learning rate must be positive")
        
        return issues
    
    def get_config_summary(self, config: ExperimentConfig) -> pd.DataFrame:
        """Get a summary of the configuration as a DataFrame."""
        def flatten_dict(d, parent_key='', sep='.'):
            items = []
            for k, v in d.items():
                new_key = f"{parent_key}{sep}{k}" if parent_key else k
                if isinstance(v, dict):
                    items.extend(flatten_dict(v, new_key, sep=sep).items())
                else:
                    items.append((new_key, v))
            return dict(items)
        
        config_dict = asdict(config)
        flat_dict = flatten_dict(config_dict)
        
        df = pd.DataFrame(list(flat_dict.items()), columns=['Parameter', 'Value'])
        return df

# Initialize configuration manager
config_manager = ConfigurationManager()

## Interactive Configuration Builder

Create and modify configurations interactively using widgets.

In [ ]:
def create_interactive_config_builder():
    """Create an interactive configuration builder using ipywidgets."""
    
    # Model configuration widgets
    encoder_type = widgets.Dropdown(
        options=['roberta', 'bert', 'deberta'],
        value='roberta',
        description='Encoder:'
    )
    
    model_name = widgets.Dropdown(
        options={
            'RoBERTa Base': 'roberta-base',
            'BERT Base': 'bert-base-uncased',
            'DeBERTa Base': 'microsoft/deberta-base'
        },
        value='roberta-base',
        description='Model:'
    )
    
    # Training configuration widgets
    batch_size = widgets.IntSlider(
        value=8, min=1, max=128, step=1,
        description='Batch Size:'
    )
    
    learning_rate = widgets.FloatLogSlider(
        value=2e-5, base=10, min=-6, max=-3,
        description='Learning Rate:'
    )
    
    max_epochs = widgets.IntSlider(
        value=100, min=1, max=500, step=1,
        description='Max Epochs:'
    )
    
    # Data configuration widgets
    max_length = widgets.Dropdown(
        options=[128, 256, 384, 512],
        value=256,
        description='Max Length:'
    )
    
    # Advanced options
    use_lora = widgets.Checkbox(
        value=False,
        description='Use LoRA'
    )
    
    gradient_checkpointing = widgets.Checkbox(
        value=True,
        description='Gradient Checkpointing'
    )
    
    auto_resume = widgets.Checkbox(
        value=True,
        description='Auto Resume'
    )
    
    # Configuration name
    config_name = widgets.Text(
        value='default_config',
        description='Config Name:'
    )
    
    # Build configuration button
    build_button = widgets.Button(
        description='Build Configuration',
        button_style='success'
    )
    
    # Output area
    output = widgets.Output()
    
    def on_build_clicked(b):
        with output:
            output.clear_output()
            
            # Create configuration
            config = ExperimentConfig()
            
            # Update model configuration
            config.model.encoder.type = encoder_type.value
            config.model.encoder.pretrained_model_name_or_path = model_name.value
            config.model.encoder.gradient_checkpointing = gradient_checkpointing.value
            config.model.encoder.lora.enabled = use_lora.value
            
            # Update training configuration
            config.training.batch_size = batch_size.value
            config.training.optimizer.learning_rate = learning_rate.value
            config.training.max_epochs = max_epochs.value
            config.training.auto_resume = auto_resume.value
            
            # Update data configuration
            config.data.max_length = max_length.value
            
            # Validate configuration
            issues = config_manager.validate_config(config)
            if issues:
                print("Configuration Issues:")
                for issue in issues:
                    print(f"  - {issue}")
                print()
            
            # Save configuration
            config_manager.save_config(config, config_name.value)
            config_manager.current_config = config
            
            # Display summary
            print("Configuration Summary:")
            summary_df = config_manager.get_config_summary(config)
            display(summary_df.head(20))  # Show first 20 parameters
    
    build_button.on_click(on_build_clicked)
    
    # Layout
    model_box = widgets.VBox([
        widgets.HTML("<h3>Model Configuration</h3>"),
        encoder_type, model_name, use_lora, gradient_checkpointing
    ])
    
    training_box = widgets.VBox([
        widgets.HTML("<h3>Training Configuration</h3>"),
        batch_size, learning_rate, max_epochs, auto_resume
    ])
    
    data_box = widgets.VBox([
        widgets.HTML("<h3>Data Configuration</h3>"),
        max_length
    ])
    
    config_box = widgets.VBox([
        widgets.HTML("<h3>Save Configuration</h3>"),
        config_name, build_button
    ])
    
    main_layout = widgets.HBox([
        widgets.VBox([model_box, data_box]),
        widgets.VBox([training_box, config_box])
    ])
    
    return widgets.VBox([main_layout, output])

# Display the interactive configuration builder
print("Interactive Configuration Builder")
print("=" * 40)
config_builder = create_interactive_config_builder()
display(config_builder)

## Configuration Presets

Pre-defined configurations for common use cases.

In [ ]:
def create_preset_configs():
    """Create and save preset configurations."""
    
    # Quick training preset
    quick_config = ExperimentConfig()
    quick_config.training.max_epochs = 10
    quick_config.training.batch_size = 16
    quick_config.training.early_stopping.patience = 2
    quick_config.data.max_length = 128
    config_manager.save_config(quick_config, "quick_training")
    
    # High-performance preset
    hp_config = ExperimentConfig()
    hp_config.model.encoder.type = "deberta"
    hp_config.model.encoder.pretrained_model_name_or_path = "microsoft/deberta-base"
    hp_config.training.batch_size = 32
    hp_config.training.optimizer.learning_rate = 1e-5
    hp_config.training.max_epochs = 50
    hp_config.data.max_length = 384
    config_manager.save_config(hp_config, "high_performance")
    
    # LoRA fine-tuning preset
    lora_config = ExperimentConfig()
    lora_config.model.encoder.lora.enabled = True
    lora_config.model.encoder.lora.r = 16
    lora_config.model.encoder.lora.alpha = 32
    lora_config.training.batch_size = 64
    lora_config.training.optimizer.learning_rate = 5e-4
    config_manager.save_config(lora_config, "lora_finetuning")
    
    # HPO preset
    hpo_config = ExperimentConfig()
    hpo_config.hpo.n_trials = 100
    hpo_config.hpo.n_jobs = 2
    hpo_config.training.max_epochs = 20
    hpo_config.training.early_stopping.patience = 2
    config_manager.save_config(hpo_config, "hpo_search")
    
    print("Created preset configurations:")
    for config_name in ["quick_training", "high_performance", "lora_finetuning", "hpo_search"]:
        print(f"  - {config_name}")

# Create preset configurations
create_preset_configs()

## Configuration Utilities

Utility functions for working with configurations.

In [ ]:
def load_and_display_config(config_name: str):
    """Load and display a configuration."""
    try:
        config = config_manager.load_config(config_name)
        print(f"Loaded configuration: {config_name}")
        print("=" * 50)
        
        # Display summary
        summary_df = config_manager.get_config_summary(config)
        display(summary_df)
        
        # Validate
        issues = config_manager.validate_config(config)
        if issues:
            print("\nValidation Issues:")
            for issue in issues:
                print(f"  ⚠️  {issue}")
        else:
            print("\n✅ Configuration is valid")
            
        return config
    except FileNotFoundError as e:
        print(f"❌ {e}")
        return None

def compare_configs(config1_name: str, config2_name: str):
    """Compare two configurations."""
    try:
        config1 = config_manager.load_config(config1_name)
        config2 = config_manager.load_config(config2_name)
        
        df1 = config_manager.get_config_summary(config1)
        df2 = config_manager.get_config_summary(config2)
        
        # Merge dataframes for comparison
        comparison = df1.merge(df2, on='Parameter', suffixes=(f'_{config1_name}', f'_{config2_name}'))
        
        # Find differences
        differences = comparison[comparison[f'Value_{config1_name}'] != comparison[f'Value_{config2_name}']]
        
        if len(differences) > 0:
            print(f"Differences between {config1_name} and {config2_name}:")
            display(differences)
        else:
            print(f"No differences found between {config1_name} and {config2_name}")
            
    except FileNotFoundError as e:
        print(f"❌ {e}")

def list_available_configs():
    """List all available configurations."""
    configs = config_manager.list_configs()
    if configs:
        print("Available configurations:")
        for config in configs:
            print(f"  📄 {config}")
    else:
        print("No configurations found")
    return configs

# Display available configurations
list_available_configs()

## Export Configuration

Export configuration to different formats for use in other notebooks.

In [ ]:
def export_config_for_training(config_name: str) -> ExperimentConfig:
    """Export configuration for use in training notebook."""
    config = config_manager.load_config(config_name)
    print(f"✅ Configuration '{config_name}' ready for training")
    print(f"   Model: {config.model.encoder.type} ({config.model.encoder.pretrained_model_name_or_path})")
    print(f"   Batch size: {config.training.batch_size}")
    print(f"   Learning rate: {config.training.optimizer.learning_rate}")
    print(f"   Max epochs: {config.training.max_epochs}")
    print(f"   Auto-resume: {config.training.auto_resume}")
    return config

def export_config_for_hpo(config_name: str) -> ExperimentConfig:
    """Export configuration for use in HPO notebook."""
    config = config_manager.load_config(config_name)
    print(f"✅ Configuration '{config_name}' ready for HPO")
    print(f"   Study: {config.hpo.study_name}")
    print(f"   Trials: {config.hpo.n_trials}")
    print(f"   Jobs: {config.hpo.n_jobs}")
    print(f"   Storage: {config.hpo.storage}")
    print(f"   Auto-resume: {config.hpo.auto_resume}")
    return config

# Example usage
print("Configuration Management Setup Complete!")
print("\nTo use configurations in other notebooks:")
print("1. Run this notebook to set up configurations")
print("2. Use export_config_for_training('config_name') in training notebook")
print("3. Use export_config_for_hpo('config_name') in HPO notebook")